In [72]:
#Import libraries
import pandas as pd
import undetected_chromedriver as uc
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

#Set up driver
driver = uc.Chrome()

In [77]:
#Function to crawl a single motel info
def single_motel_info(item):
    driver.find_elements(By.CLASS_NAME, 'pr-container')[item].click()
    time.sleep(3)
    
    #Get price info
    price = driver.find_elements(By.CLASS_NAME, 're__pr-short-info-item')[0].text.split('\n')[1]
    if ',' in price.split(' ')[0]:
        price_by_vnd = int(price.split(' ')[0].replace(',',''))*100000
    elif 'Th' in price.split(' ')[0]:
        price_by_vnd = ''
    else:
        price_by_vnd = int(price.split(' ')[0])*1000000

    #Get area info
    area = driver.find_elements(By.CLASS_NAME, 're__pr-short-info-item')[1].text.split('\n')[1]
    area_by_square_meter = int(area.split(' ')[0])

    #Get address info
    address_detail = driver.find_element(By.CLASS_NAME, 'js__pr-address').get_attribute('innerHTML').split(',')
    ##Get street name
    street_ext = address_detail[0:-3]
    street = ''.join(street_ext)
    ##Get ward name
    ward_ext = address_detail[-3].strip()
    if 'P.' in ward_ext or 'Ph' in ward_ext:
        new_ward_ext = ward_ext.split(' ')[1:]
        ward = ' '.join(new_ward_ext)
    else:
        ward = ward_ext
    ##Get district name
    dist_ext = address_detail[-2].strip()
    if 'Q.' in dist_ext or 'Qu' in dist_ext:
        new_dist_ext = dist_ext.split(' ')[1:]
        dist = ' '.join(new_dist_ext)
    else:
        dist = dist_ext
    ##Get city name
    city = address_detail[-1].strip()
    
    #Get motel link
    link = driver.current_url
    
    #Wrap up all info
    motel_data = {'City':city, 'District':dist, 'Price_by_vnd':price_by_vnd, 'Area_by_square_meter':area_by_square_meter, 'Street_name': street, 'Ward': ward, 'Link': link}
    return motel_data

#Function to crawl motel info in full page
motel_data_by_number_of_page = []

def motel_info_by_number_of_page(page_url, page_start, page_end): #Choose 1 number for both page_start & page_end if you want to crawl only 1 page
    for i in range(page_start, page_end+1):
        new_page_url = f'{page_url}/p{i}'
        driver.get(new_page_url)
        time.sleep(10) #Increase number if you have a slow internet connection
        number_of_motel_in_page = driver.find_elements(By.CLASS_NAME, 'pr-container')
        for item in range(len(number_of_motel_in_page)):
            single_motel_data = single_motel_info(item)
            driver.back()
            motel_data_by_number_of_page.append(single_motel_data)
            driver.implicitly_wait(10) #Increase number if you have a slow internet connection
    return motel_data_by_number_of_page

motel_info_by_number_of_page('https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-tp-hcm',3,3)

[{'City': 'Hồ Chí Minh',
  'District': '7',
  'Price_by_vnd': 4700000,
  'Area_by_square_meter': 28,
  'Street_name': 'Him Lam Kênh Tẻ 23 Đường 14',
  'Ward': 'Tân Hưng',
  'Link': 'https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-14-1-phuong-tan-hung-14-prj-khu-do-thi-him-lam-kenh-te/cho-can-ho-dich-vu-28m2-hi-quan-7-pr38082984'},
 {'City': 'Hồ Chí Minh',
  'District': '6',
  'Price_by_vnd': 2600000,
  'Area_by_square_meter': 30,
  'Street_name': '195 Đường Bà Hom',
  'Ward': '13',
  'Link': 'https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-ba-hom-phuong-13-3/-rat-rong-30m2-co-gac-lung-ke-bep-gio-tu-do-vo-trung-tam-chi-10-phut-pr37325141'},
 {'City': 'Hồ Chí Minh',
  'District': 'Bình Tân',
  'Price_by_vnd': 2800000,
  'Area_by_square_meter': 25,
  'Street_name': '24/32 Đường Trương Phước Phan',
  'Ward': 'Bình Trị Đông',
  'Link': 'https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-truong-phuoc-phan-phuong-binh-tri-dong/-moi-tinh-dang-cap-may-lanh-tu-ao-tu

In [83]:
#Convert into dataframe & Double check data
df = pd.DataFrame(data = motel_data_by_number_of_pages, columns = motel_data_by_number_of_pages[0].keys())
number_of_row = int(input('Number of row you want to view: '))
df.head(number_of_row)

Number of row you want to view: 5


,City,District,Price_by_vnd,Area_by_square_meter,Street_name,Ward,Link
0,Hồ Chí Minh,7,4700000,28,Him Lam Kênh Tẻ 23 Đường 14,Tân Hưng,https://batdongsan.com.vn/cho-thue-nha-tro-pho...
1,Hồ Chí Minh,6,2600000,30,195 Đường Bà Hom,13,https://batdongsan.com.vn/cho-thue-nha-tro-pho...
2,Hồ Chí Minh,Bình Tân,2800000,25,24/32 Đường Trương Phước Phan,Bình Trị Đông,https://batdongsan.com.vn/cho-thue-nha-tro-pho...
3,Hồ Chí Minh,Gò Vấp,4000000,27,Đường Số 5,17,https://batdongsan.com.vn/cho-thue-nha-tro-pho...
4,Hồ Chí Minh,Tân Bình,4600000,20,Đường Xuân Diệu,4,https://batdongsan.com.vn/cho-thue-nha-tro-pho...


In [71]:
#Export data in csv
df.to_csv('motel_data_by_number_of_pages.csv', index=False, encoding='utf-8-sig') #Change a file name if you want